In [9]:
import torchaudio
import torch

 10-second audio waveform -> sequence of 128-dimensional long filterbank 
 -> 1024(time) x 128(frequency) spectogram
 split into 512(64 time) x 8(frequency)) square patches of shape 16x16 fed into AST

In [10]:

# Filterbank
def load_audio(filename):
    waveform, sr = torchaudio.load(filename)
    waveform = waveform - waveform.mean()
    
    # 128 dimensional long filterbank
    fbank = torchaudio.compliance.kaldi.fbank(waveform, htk_compat=True, sample_frequency=sr,
                                              use_energy=False, window_type='hanning',
                                              num_mel_bins=128, dither=0.0, frame_shift=10)
    target_length = 1024
    n_frames = fbank.shape[0]
    p = target_length - n_frames
    if p > 0:
        m = torch.nn.ZeroPad2d((0, 0, 0, p))
        fbank = m(fbank)
    elif p < 0:
        fbank = fbank[0:target_length, :]
    # normalize the fbank
    fbank = (fbank + 5.081) / 4.4849

    # returns a 1024x128 spectogram
    return fbank

In [11]:
file = "./data/sample/audio_samples/audio/650981.flac"
fb = load_audio(file)

In [12]:
fb.shape

torch.Size([1024, 128])